In [47]:
#pip install pandas langchain openai tiktoken

from langchain_core.documents import Document

In [86]:
import pandas as pd
from pathlib import Path

# 1. 입력 CSV 경로들
CSV_FILES = [
    "data/processed/law/law_labeled.csv",
    "data/processed/rule/rule_labeled.csv",
    "data/processed/case/case_labeled.csv",
]

# 2. CSV 로드
dfs = []
for path in CSV_FILES:
    df = pd.read_csv(path)
    dfs.append(df)

# 3. 하나로 병합
merged_df = pd.concat(dfs, ignore_index=True)

# 4. (선택) id 중복 체크
dup_ids = merged_df["id"].duplicated().sum()
print(f"중복 id 개수: {dup_ids}")

# 5. 저장
output_path = Path("data/processed/all_chunks.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

merged_df.to_csv(output_path, index=False)

print(f"✅ 병합 완료: {output_path}")
print(f"총 청크 수: {len(merged_df)}")

중복 id 개수: 0
✅ 병합 완료: data\processed\all_chunks.csv
총 청크 수: 248


In [90]:
import pandas as pd
from langchain_core.documents import Document

def infer_law_type(source: str) -> str:
    if "주택임대차보호법" in source:
        return "주택임대차보호법"
    if "상가건물 임대차보호법" in source:
        return "상가건물임대차보호법"
    if "민법" in source:
        return "민법"
    if "시행령" in source:
        return "시행령"
    if "규칙" in source:
        return "규칙"
    if "사례" in source or source.endswith(".pdf"):
        return "사례"
    return "기타"

def load_chunked_csv(csv_path: str):
    df = pd.read_csv(csv_path)

    documents = []
    for _, row in df.iterrows():
        documents.append(
            Document(
                page_content=row["text"],
                metadata={
                    "id": row["id"],
                    "category": row["category"],
                    "law_type": infer_law_type(row["source"]),  # ⭐ 자동 생성
                    "priority": int(row["priority"]),
                    "article": row["article"],
                    "chunk_id": row["chunk_id"],
                    "source": row["source"],
                    "text": row["text"],  # ⭐ Pinecone에서 쓰기 위함
                }
            )
        )
    return documents

In [91]:
docs = load_chunked_csv("data/processed/all_chunks.csv")

In [92]:
docs[0]

Document(metadata={'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f', 'category': 'law', 'law_type': '민법', 'priority': 4, 'article': '제390조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx', 'text': '(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'}, page_content='(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차')

In [93]:
docs[0].page_content

'(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'

In [94]:
docs[0].page_content[:300]

'(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'

In [112]:
for i in range(5):
    print(f"\n--- doc {i} ---")
    print(docs[i].metadata)
    print(docs[i].page_content[:200])


--- doc 0 ---
{'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f', 'category': 'law', 'law_type': '민법', 'priority': 4, 'article': '제390조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx', 'text': '(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'}
(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.
제2장 계약
제6절 사용대차

--- doc 1 ---
{'id': '767cb115-6be6-441b-a624-0de4fdd5d05d', 'category': 'law', 'law_type': '민법', 'priority': 4, 'article': '제618조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx', 'text': '(임대차의 의의) 임대차는 당사자 일방이 상대방에게 목적물을 사용, 수익하게 할 것을 약정하고 상대방이 이에 대하여 차임을 지급할 것을 약정함으로써 그 효력이 생긴다.'}
(임대차의 의의) 임대차는 당사자 일방이 상대방에게 목적물을 사용, 수익하게 할 것을 약정하고 상대방이 이에 대하여 차임을 지급할 것을 약정함으로써 그 효력이 생긴다.

--- doc 2 ---
{'id': '1bb029e6-f430-4d8e-8777-f93e7ac57c90', 'category': 'law', 'law_type': '민법', 'priority': 4, 'article': '제619조', 'chu

In [113]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [114]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

texts = [doc.page_content for doc in docs]

vectors = embeddings.embed_documents(texts)

print(f"임베딩 완료: {len(vectors)}")
print(f"벡터 차원: {len(vectors[0])}")

임베딩 완료: 248
벡터 차원: 3072


In [115]:
import numpy as np

np.save(
    "data/processed/law_vectors.npy",
    np.array(vectors)
)

In [116]:
# import json

# metadata = [doc.metadata for doc in docs]

# with open(
#     "data/embeddings/law_metadata.json",
#     "w",
#     encoding="utf-8"
# ) as f:
#     json.dump(metadata, f, ensure_ascii=False, indent=2)

In [117]:
embedded_items = []

for doc, vector in zip(docs, vectors):
    embedded_items.append({
        "id": doc.metadata["id"],
        "vector": vector,
        "text": doc.page_content,
        "metadata": doc.metadata
    })

embedded_items[0]

{'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f',
 'vector': [0.005004982929676771,
  -0.04791807383298874,
  -0.009500198997557163,
  0.027527404949069023,
  -0.0030730825383216143,
  0.01639363542199135,
  -0.03688857704401016,
  -0.0016523973317816854,
  -0.03776908293366432,
  -0.01238501537591219,
  0.020228471606969833,
  0.0010101202642545104,
  -0.01810830645263195,
  -0.012883196584880352,
  -0.009998380206525326,
  -0.02537248097360134,
  -0.018664414063096046,
  0.01891929842531681,
  -0.019811389967799187,
  0.0024749755393713713,
  -0.03776908293366432,
  0.02402855083346367,
  0.015432029962539673,
  -0.0071888696402311325,
  -0.03848738968372345,
  0.007658087182790041,
  0.0027269625570625067,
  0.017818665131926537,
  0.0004170819011051208,
  -0.02490905672311783,
  -0.026299331337213516,
  0.008451701141893864,
  -0.03957643732428551,
  -0.005132424179464579,
  0.007692843675613403,
  -0.027110323309898376,
  -0.008492250926792622,
  -0.021027877926826477,
  -0.011579815

In [118]:
pinecone_vectors = [
    {
        "id": item["id"],
        "values": item["vector"],
        "metadata": item["metadata"]
    }
    for item in embedded_items
]

# PINECONE


In [119]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

index = pc.Index("realestate")  # 네 인덱스 이름

In [120]:
pinecone_vectors = []

for item in embedded_items:
    pinecone_vectors.append({
        "id": item["id"],
        "values": item["vector"],
        "metadata": item["metadata"]
    })

In [121]:
BATCH_SIZE = 100

for i in range(0, len(pinecone_vectors), BATCH_SIZE):
    batch = pinecone_vectors[i:i + BATCH_SIZE]
    index.upsert(vectors=batch)
    print(f"{i} ~ {i + len(batch)} 업로드 완료")

0 ~ 100 업로드 완료
100 ~ 200 업로드 완료
200 ~ 248 업로드 완료


In [122]:
res = index.query(
    vector=embedded_items[0]["vector"],
    top_k=5,
    include_metadata=True
)

for m in res["matches"]:
    print(m["score"], m["metadata"].get("article"))

1.00032246 제390조
0.577142775 제750조
0.491048843 제6조의2
0.457052261 사례 1
0.445214301 제617조


# 임베딩 결과 확인


In [125]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
INDEX_NAME = "realestate"   # 지금은 하드코딩 권장

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

emb = OpenAIEmbeddings(model="text-embedding-3-large")

def test_search(query, top_k=5):
    qvec = emb.embed_query(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True
    )
    return res

In [126]:
res = test_search("전세 보증금 우선변제 요건")
for m in res["matches"]:
    md = m["metadata"]
    print(m["score"], md.get("source"), md.get("article"))

0.528787613 주택임대차보호법(법률)(제21065호)(20260102).docx 제3조의2
0.489928246 주택임대차보호법 시행령(대통령령)(제35947호)(20260102).docx 제3조의2
0.461048126 주택임대차보호법 시행령(대통령령)(제35947호)(20260102).docx 제22조
0.458220482 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx 제3조의2
0.456645 상가건물 임대차보호법(법률)(제21065호)(20260102).docx 제5조


# 최소 코드로 LAG 가동
